In [1]:
import pandas as pd
import numpy as np

X_Bundesliga_concede = pd.read_csv('data/X_Bundesliga_concede.csv', index_col=0)
Y_Bundesliga_concede = pd.read_csv('data/Y_Bundesliga_concede.csv', index_col=0)['concedes']
weight_Bundesliga_concede = pd.read_csv('data/weight_Bundesliga_concede.csv', index_col=0)['weight']

X_Bundesliga_concede.shape, Y_Bundesliga_concede.shape, weight_Bundesliga_concede.shape

((912958, 46), (912958,), (912958,))

In [2]:
X_LaLiga_concede = pd.read_csv('data/X_LaLiga_concede.csv', index_col=0)
Y_LaLiga_concede = pd.read_csv('data/Y_LaLiga_concede.csv', index_col=0)['concedes']
weight_LaLiga_concede = pd.read_csv('data/weight_LaLiga_concede.csv', index_col=0)['weight']

X_LaLiga_concede.shape, Y_LaLiga_concede.shape, weight_LaLiga_concede.shape

((1139048, 46), (1139048,), (1139048,))

In [3]:
X_Ligue1_concede = pd.read_csv('data/X_Ligue1_concede.csv', index_col=0)
Y_Ligue1_concede = pd.read_csv('data/Y_Ligue1_concede.csv', index_col=0)['concedes']
weight_Ligue1_concede = pd.read_csv('data/weight_Ligue1_concede.csv', index_col=0)['weight']

X_Ligue1_concede.shape, Y_Ligue1_concede.shape, weight_Ligue1_concede.shape

((1164060, 46), (1164060,), (1164060,))

In [4]:
X_PremierLeague_concede = pd.read_csv('data/X_PremierLeague_concede.csv', index_col=0)
Y_PremierLeague_concede = pd.read_csv('data/Y_PremierLeague_concede.csv', index_col=0)['concedes']
weight_PremierLeague_concede = pd.read_csv('data/weight_PremierLeague_concede.csv', index_col=0)['weight']

X_PremierLeague_concede.shape, Y_PremierLeague_concede.shape, weight_PremierLeague_concede.shape

((1155085, 46), (1155085,), (1155085,))

In [5]:
X_SerieA_concede = pd.read_csv('data/X_SerieA_concede.csv', index_col=0)
Y_SerieA_concede = pd.read_csv('data/Y_SerieA_concede.csv', index_col=0)['concedes']
weight_SerieA_concede = pd.read_csv('data/weight_SerieA_concede.csv', index_col=0)['weight']

X_SerieA_concede.shape, Y_SerieA_concede.shape, weight_SerieA_concede.shape

((1162685, 46), (1162685,), (1162685,))

### 拟合模型

In [ ]:
#!pip install xgboost

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
  Using cached https://mirrors.cloud.aliyuncs.com/pypi/packages/63/f1/653afe1a1b7e1d03f26fd4bd30f3eebcfac2d8982e1a85b6be3355dcae25/xgboost-3.0.0-py3-none-manylinux_2_28_x86_64.whl (253.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 13.3 MB/s eta 0:00:0000:0100:01
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a co

In [7]:
from sklearn.metrics import brier_score_loss
import xgboost

X_concede = pd.concat([X_Bundesliga_concede, X_LaLiga_concede, X_Ligue1_concede, X_PremierLeague_concede, X_SerieA_concede], ignore_index=True)
Y_concede = pd.concat([Y_Bundesliga_concede, Y_LaLiga_concede, Y_Ligue1_concede, Y_PremierLeague_concede, Y_SerieA_concede], ignore_index=True)
weight_concede = pd.concat([weight_Bundesliga_concede, weight_LaLiga_concede, weight_Ligue1_concede, weight_PremierLeague_concede, weight_SerieA_concede], ignore_index=True)
folds_concede = [0] * len(X_Bundesliga_concede) + [1] * len(X_LaLiga_concede) + [2] * len(X_Ligue1_concede) + [3] * len(X_PremierLeague_concede) + [4] * len(X_SerieA_concede)
folds_concede = np.array(folds_concede)

del X_Bundesliga_concede, X_LaLiga_concede, X_Ligue1_concede, X_PremierLeague_concede, X_SerieA_concede
del Y_Bundesliga_concede, Y_LaLiga_concede, Y_Ligue1_concede, Y_PremierLeague_concede, Y_SerieA_concede
del weight_Bundesliga_concede, weight_LaLiga_concede, weight_Ligue1_concede, weight_PremierLeague_concede, weight_SerieA_concede

categorical_features = ['period_id_a0', 'period_id_a1', 'period_id_a2', 'actiontype_a0', 'actiontype_a1', 'actiontype_a2', 'bodypart_a0', 'bodypart_a1', 'bodypart_a2']
for col in categorical_features:
    if X_concede[col].dtype != 'category':
        # print(col, X_concede[col].dtype)
        X_concede[col] = X_concede[col].astype('category')
        
X_concede.shape, Y_concede.shape, weight_concede.shape, folds_concede

((5533836, 46), (5533836,), (5533836,), array([0, 0, 0, ..., 4, 4, 4]))

In [8]:
model_params_list = [
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True}
]

best_params, best_mean_brier = 0, 1

for model_params in model_params_list:

    models = [xgboost.XGBClassifier(**model_params)] * 5
    briers = np.zeros(5)

    for i in range(5):
        train_indices = (folds_concede != i)
        test_indices = (folds_concede == i)
        models[i].fit(X_concede[train_indices], Y_concede[train_indices], sample_weight=weight_concede[train_indices])
        briers[i] = brier_score_loss(Y_concede[test_indices],
                                     models[i].predict_proba(X_concede[test_indices])[:, 1],
                                     sample_weight=weight_concede[test_indices])

    print(model_params)
    print("Test brier score per fold:", briers)
    print("Mean test brier score:", briers.mean())
    
    if briers.mean() < best_mean_brier: best_params, best_mean_brier = model_params, briers.mean()
    
print('Best CV brier score: ', best_mean_brier)
print('Best parameters: ', best_params)

model_concede = xgboost.XGBClassifier(**best_params)
model_concede.fit(X_concede, Y_concede, sample_weight=weight_concede)

[17:13:03] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4620878, 46, 212560388).
[17:13:58] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4394788, 46, 202160248).
[17:14:48] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4369776, 46, 201009696).
[17:15:39] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4378751, 46, 201422546).
[17:16:30] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4371151, 46, 201072946).
{'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0, 'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True}
Test brier score per fold: [0.00099392 0.00100589 0.0009488  0.00094819 0.00092723]
Mean test brier score: 0.0009648083990016268
[17:17:21] INFO: /workspace/src/data/iterativ

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_parallel_tree=None, ...)

In [9]:
feature_concede = pd.DataFrame({'feature': model_concede.feature_names_in_, 'importance': model_concede.feature_importances_})
feature_concede.sort_values('importance', ascending=False)

,feature,importance
40,actiontype_a0,0.503751
41,actiontype_a1,0.069205
13,x_a0,0.061479
11,time_delta_1,0.043963
20,angle_to_goal_a0,0.041215
15,x_a1,0.026900
25,mov_d_a0,0.019456
31,dx_a0,0.011981
42,actiontype_a2,0.010774
10,team_2,0.010141


### 保存模型

In [10]:
import joblib
joblib.dump(model_concede, 'data/model_concede.dat')

['data/model_concede.dat']